In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium 
import io
import requests
from bs4 import BeautifulSoup

print('Done')

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20181022092127-0000
Solving environment: done

## Package Plan ##

  environment location: /opt/ibm/conda/miniconda3

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    conda-4.5.11               |           py35_0         636 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.1 MB

The followin

#### Getting  Toronto data

In [2]:


source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')
table = soup.find('table', class_='wikitable sortable')
table_rows = table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
        
df = pd.DataFrame(res, columns=["Postcode", "Borough", "Neighbourhood"])
df = df[df.Borough != 'Not assigned']
idx=df.index[df.Neighbourhood == 'Not assigned']
df['Neighbourhood'] = df['Borough'].where(df['Neighbourhood']=='Not assigned', df['Neighbourhood'])
df.loc[idx]
df=df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

url="https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv"
file=requests.get(url).content
df2=pd.read_csv(io.StringIO(file.decode('utf-8')))
df2.columns = ['Postcode', 'Latitude','Longitude']
df1 = pd.merge(df, df2, on='Postcode')
df1.head()



,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Find the latitude and longitude of Toronto

In [3]:
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print((latitude, longitude))

/opt/ibm/conda/miniconda3/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


(43.653963, -79.387207)


#### Get the boroughs with the word Toronto in it


In [4]:
borough_with_Toronto = df1[df1['Borough'].str.contains("Toronto")].reset_index(drop=True)
# borough_Toronto_data = df1
print(borough_with_Toronto.shape)
borough_with_Toronto.head()

(38, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


#### Create a map with Toronto on it

In [5]:
map_boroughs_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(borough_with_Toronto['Latitude'], borough_with_Toronto['Longitude'], borough_with_Toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_boroughs_Toronto)  
    
map_boroughs_Toronto

#### Connect to Foursquare

In [6]:
CLIENT_ID = 'VI1HZZN4HJQHMPBKYZUQ52OQVJFP3LLJ03ITAIQXTMEC3S1M' # your Foursquare ID
CLIENT_SECRET = 'EJ2C0NQ31QONYYFULJLYDIOV4NB22WI1TK4XDUQEXFKVVM2V' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
# type your answer here
LIMIT = 300 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

Your credentails:
CLIENT_ID: VI1HZZN4HJQHMPBKYZUQ52OQVJFP3LLJ03ITAIQXTMEC3S1M
CLIENT_SECRET:EJ2C0NQ31QONYYFULJLYDIOV4NB22WI1TK4XDUQEXFKVVM2V


In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Toronto_venues = getNearbyVenues(names=borough_with_Toronto['Neighbourhood'],
                                   latitudes=borough_with_Toronto['Latitude'],
                                   longitudes=borough_with_Toronto['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [8]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

print(Toronto_venues.shape)
Toronto_venues.head()

(1705, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
3,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
4,"The Danforth West, Riverdale",43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop


In [9]:
Toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,53,53,53,53,53,53
"Brockton, Exhibition Place, Parkdale Village",21,21,21,21,21,21
Business reply mail Processing Centre969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",48,48,48,48,48,48
Central Bay Street,82,82,82,82,82,82
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


In [10]:
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighbourhood'] 

fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.shape

Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
Toronto_grouped.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.01,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.0,0.0,0.0,0.010000,0.01,0.0,0.02,0.0,0.000000,0.0,0.0,0.000000,0.020000,0.0,0.030000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.01,0.000000,0.0,0.01,0.020000,0.000000,0.0,0.0,0.020000,0.010000,0.0,0.000000,0.060000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.020000,0.010000,0.070000,0.0,0.0,0.0,0.000000,0.000000,0.020000,0.000000,0.030000,0.0,0.000000,0.0,0.0,0.0,0.02,0.01,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.01,0.0

In [11]:
# Filter on Italian Restaurants
Italian = Toronto_grouped.loc[:, ['Neighbourhood', 'Italian Restaurant']]
Italian. sort_values(by=['Italian Restaurant'], ascending=False)

,Neighbourhood,Italian Restaurant
8,Christie,0.062500
29,"Runnymede, Swansea",0.055556
11,Davisville,0.052632
36,"The Beaches West, India Bazaar",0.050000
33,Studio District,0.048780
6,Central Bay Street,0.048780
37,"The Danforth West, Riverdale",0.047619
2,"Brockton, Exhibition Place, Parkdale Village",0.047619
5,"Cabbagetown, St. James Town",0.041667
21,"High Park, The Junction South",0.041667


In [12]:
# set number of clusters
kclusters = 8

Toronto_grouped_clustering = Italian.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

Toronto_merged = borough_with_Toronto

# add clustering labels
Toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(Italian.set_index('Neighbourhood'), on='Neighbourhood')

Toronto_merged.sort_values(by=['Italian Restaurant'], ascending=False)

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,Italian Restaurant
30,M6G,Downtown Toronto,Christie,43.669542,-79.422564,4,0.062500
36,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,5,0.055556
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,0.052632
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,5,0.050000
3,M4M,East Toronto,Studio District,43.659526,-79.340923,3,0.048780
17,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,3,0.048780
33,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191,5,0.047619
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,4,0.047619
34,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763,3,0.041667
11,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,1,0.041667


In [13]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
#x = np.arange('Italian Restaurant')
#ys = [i+x+(i*x)**2 for i in range('Italian Restaurant')]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

NameError: name 'rainbow' is not defined

In [14]:
Toronto_merged['marker_color'] = pd.cut(Toronto_merged['Italian Restaurant'], bins=4, labels =['blue','green','yellow','red'])

In [15]:
map_frequency = folium.Map(location=[latitude, longitude], zoom_start=13)

for index, row in Toronto_merged.iterrows():
    folium.CircleMarker([row['Latitude'], row['Longitude']],
                    radius=15, color=row['marker_color']).add_to(map_frequency)
    
map_frequency

In [16]:
Toronto_merged.sort_values(by=['Italian Restaurant'], ascending=False)

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,Italian Restaurant,marker_color
30,M6G,Downtown Toronto,Christie,43.669542,-79.422564,4,0.062500,red
36,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,5,0.055556,red
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,0.052632,red
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,5,0.050000,red
3,M4M,East Toronto,Studio District,43.659526,-79.340923,3,0.048780,red
17,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,3,0.048780,red
33,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191,5,0.047619,red
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,4,0.047619,red
34,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763,3,0.041667,yellow
11,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,1,0.041667,yellow


In [17]:
test = Toronto_merged[Toronto_merged.marker_color != 'red']
test = test[test.marker_color != 'yellow']
test.sort_values(by=['Italian Restaurant'], ascending=False)
#df = df[df.Borough != 'Not assigned']


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,Italian Restaurant,marker_color
20,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,4,0.030000,green
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,0.030000,green
21,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,7,0.030000,green
19,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,2,0.030000,green
25,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049,3,0.027778,green
28,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846,3,0.021739,green
29,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,1,0.020000,green
14,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,2,0.020000,green
13,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,3,0.020000,green
16,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,4,0.018868,green
